In [253]:
## Imports
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from datetime import datetime
import os
import urllib.request
import nltk
nltk.download("wordnet")

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', "organization", "receive"])

try:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
except Exception as e:
    os.system("python -m spacy download en")
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aantonison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.

In [254]:
def download_project_maslow_files():
    """The purpose of this function is to handle getting the nonprofit.txt and nonprofit_text.txt
    files.  This functin will create a data directory, add a .gitignore file so
    source control does not pick up the txt files, and then check to see
    if either nonprofit.txt and nonprofit_text.txt have been downloaded yet.
    If not, it will proceed to download the files.
    """

    if os.path.isdir("data") is False:
        print("data/ folder does not exist, creating it now")
        os.mkdir("data")

    if os.path.isfile("data/.gitignore") is False:
        os.system("echo *.txt > data/.gitignore")

    if os.path.isfile("data/nonprofit.txt") is False:
        print("Downloading nonprofit.txt...")
        urllib.request.urlretrieve("https://mtsu-dsi-hackathon-2022.s3.amazonaws.com/nonprofit.txt", 'data/nonprofit.txt')

    if os.path.isfile("data/nonprofit_text.txt") is False:
        print("Downloading nonprofit_text.txt...")
        urllib.request.urlretrieve("https://mtsu-dsi-hackathon-2022.s3.amazonaws.com/nonprofit_text.txt", 'data/nonprofit_text.txt')

In [255]:
def get_non_profit_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit
    """

    download_project_maslow_files()

    col_types = {
        "nonprofit_id": "Int64",
        "reporting_year": "Int64",
        "ein": "Int64",
        "businessname": "str",
        "phone": "str",
        "address1": "str",
        "address2": "str",
        "city": "str",
        "stabbrv": "str",
        "zip": "str"
      }

    df = pd.read_csv("data/nonprofit.txt", sep = "|", dtype=col_types)

    return df
df_np = get_non_profit_df()
df_np = df_np[["nonprofit_id", "stabbrv"]]


In [256]:
def get_non_profit_text_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit_text
    """

    download_project_maslow_files()

    col_types = {
        "nonprofit_text_id": "Int64",
        "reporting_year": "Int64",
        "nonprofit_id": "Int64",
        "grouptype": "str",
        "description": "str"
    }

    df = pd.read_csv("data/nonprofit_text.txt", sep = "|", encoding="cp1252", dtype=col_types)

    return df

np_text_df = get_non_profit_text_df()

In [290]:
def preprocess_non_profit_descriptions(df_np, df_np_text):
    # merge onto the nonprofit dataset and filter down to just TN
    merged_df = np_text_df.merge(df_np, on="nonprofit_id")
    filtered_df = merged_df[merged_df["stabbrv"] == "TN"].copy(deep=True)
    filtered_df = filtered_df[["nonprofit_id", "description"]]

    # remove instances where the description is duplicated across
    # multiple categories
    de_dup_df = filtered_df.drop_duplicates().copy(deep=True)
    de_dup_df["description_word_length"] = de_dup_df["description"].str.split().str.len()
    np_desc_populated = de_dup_df[de_dup_df["description_word_length"] > 20].copy(deep=True)
    np_desc_populated.head()

    # rank the different descriptions and select the longest one
    # note: if it has two descriptions that are the same length, it will arbitrarily select one of them
    np_desc_populated["description_rank"] = np_desc_populated.groupby('nonprofit_id')['description_word_length'].rank()
    max_desc_selected_df = np_desc_populated[np_desc_populated["description_rank"] == 1].copy(deep=True)
    result_df = max_desc_selected_df[["nonprofit_id", "description"]].reset_index(drop=True)

    return result_df


df_text = preprocess_non_profit_descriptions(df_np, np_text_df)

# show if there are any duplicate records
if df_text[df_text.duplicated(['nonprofit_id'])].shape[0] > 0:
    print("df_text has duplicate values")
    raise

In [262]:
# create a list of the description
data = df_text.description.values.tolist()
np.shape(data)

(4085,)

In [263]:
# tokenize and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
np.shape(data_words)

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(4085,)

In [264]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'foundation', 'primary', 'objective', 'is', 'to', 'save', 'abandoned', 'pets', 'by', 'providing', 'care', 'and', 'shelter', 'until', 'they', 'are', 'placed', 'in', 'viable', 'permanent', 'homes', 'during', 'the', 'year', 'the', 'foundation', 'was', 'involved', 'in', 'several', 'local', 'activities', 'to', 'promote', 'the', 'shelter', 'objectives', 'and', 'making', 'the', 'public', 'aware', 'of', 'the', 'shelter', 'programs', 'and', 'needs', 'hundreds', 'of', 'pets', 'were', 'placed', 'due', 'to', 'the', 'shelter', 'programs', 'the', 'costs', 'of', 'these', 'temporary', 'activities', 'are', 'small', 'compared', 'to', 'the', 'overall', 'cost', 'to', 'permanently', 'protect', 'and', 'care', 'for', 'abandoned', 'pets']


In [265]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [266]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

data_words_nostops[0]

['foundation',
 'primary',
 'objective',
 'save',
 'abandoned',
 'pets',
 'providing',
 'care',
 'shelter',
 'placed',
 'viable',
 'permanent',
 'homes',
 'year',
 'foundation',
 'involved',
 'several',
 'local',
 'activities',
 'promote',
 'shelter',
 'objectives',
 'making',
 'public',
 'aware',
 'shelter',
 'programs',
 'needs',
 'hundreds',
 'pets',
 'placed',
 'due',
 'shelter',
 'programs',
 'costs',
 'temporary',
 'activities',
 'small',
 'compared',
 'overall',
 'cost',
 'permanently',
 'protect',
 'care',
 'abandoned',
 'pets']

In [267]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams[0]

['foundation',
 'primary',
 'objective',
 'save',
 'abandoned',
 'pets',
 'providing',
 'care',
 'shelter',
 'placed',
 'viable',
 'permanent',
 'homes',
 'year',
 'foundation',
 'involved',
 'several',
 'local',
 'activities',
 'promote',
 'shelter',
 'objectives',
 'making',
 'public',
 'aware',
 'shelter',
 'programs',
 'needs',
 'hundreds',
 'pets',
 'placed',
 'due',
 'shelter',
 'programs',
 'costs',
 'temporary',
 'activities',
 'small',
 'compared',
 'overall',
 'cost',
 'permanently',
 'protect',
 'care',
 'abandoned',
 'pets']

In [268]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

data_lemmatized[0]

['foundation',
 'primary',
 'objective',
 'save',
 'abandon',
 'pet',
 'provide',
 'care',
 'shelter',
 'place',
 'viable',
 'permanent',
 'home',
 'year',
 'foundation',
 'involve',
 'several',
 'local',
 'activity',
 'promote',
 'shelter',
 'objective',
 'make',
 'public',
 'aware',
 'shelter',
 'program',
 'need',
 'hundred',
 'pet',
 'place',
 'due',
 'shelter',
 'program',
 'cost',
 'temporary',
 'activity',
 'small',
 'compare',
 'overall',
 'cost',
 'permanently',
 'protect',
 'care',
 'abandon',
 'pet']

In [269]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 2), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 3), (19, 2), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 4), (29, 1), (30, 1), (31, 1), (32, 1)]]


In [270]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [271]:
example_num = 1

In [272]:
df_text.loc[example_num,"description"]

"THE AUTISM FOUNDATION OF TENNESSEE'S PRIMARY EXEMPT PURPOSE IS TO PROVIDE LOW COST THERAPY AND SERVICES TO CHILDERN ON THE AUTISM SPECTRUM. BY CARRYING OUT THIS PURPOSE, WE WERE ABLE TO EVALUATE BEHAVIOR PROBLEMS, ACADEMIC AND DAILY LIVING SKILLS FOR EACH CHILD. THE FOUNDATION WAS ALSO ABLE TO DEVELOP A SPECIFIC OBSERVABLE AND MEASURABLE TREATMENT PLAN INDIVIDUALIZED FOR EACH CLIENT. THE PLANS WERE STRATEGICALLY IMPLEMENTED TO INCREASE APPROPRIATE BEHAVIORS AND ACADEMIC SKILLS WHILE DECREASING INAPPROPRIATE BEHAVIORS."

In [273]:
data_lemmatized[example_num]

['autism',
 'foundation',
 'tennessee',
 'primary_exempt',
 'purpose',
 'provide',
 'low',
 'cost',
 'therapy',
 'service',
 'childern',
 'autism_spectrum',
 'carry',
 'purpose',
 'able',
 'evaluate',
 'behavior',
 'problem',
 'academic',
 'daily',
 'live',
 'skill',
 'child',
 'foundation',
 'also',
 'able',
 'develop',
 'specific',
 'observable',
 'measurable',
 'treatment',
 'plan',
 'individualized',
 'client',
 'plan',
 'strategically',
 'implement',
 'increase',
 'appropriate',
 'behavior',
 'academic',
 'skill',
 'decrease',
 'inappropriate',
 'behavior']

In [274]:
doc = lda_model.id2word.doc2bow(data_lemmatized[example_num])
doc_topics = lda_model.get_document_topics(doc, per_word_topics=False)
doc_topics
for val in doc_topics:
    print(val)
    print(lda_model.id2word[val[0]])

(0, 0.20779145)
abandon
(1, 0.5941474)
activity
(2, 0.015454655)
aware
(3, 0.13022263)
care
(4, 0.016131224)
compare
(5, 0.036252603)
cost


In [275]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics = lda_model.get_document_topics(doc, per_word_topics=False)
        
        # initialize
        cur_val = 0
        cur_topic = 0

        # will loop over all of the doc topics
        # to find the one with the greatest 
        for val in doc_topics:
            if val[1] > cur_val:
                cur_val = val[1]
                cur_topic = val[0]
        lda_model.id2word[cur_topic]
        return lda_model.id2word[cur_topic]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_result"] = get_readable_topic(doc)
    index = index + 1

In [276]:
df_text["topic_result"].unique()

array(['compare', 'activity', 'abandon', 'cost', 'care', 'aware'],
      dtype=object)

In [277]:
# export model results into unique file
df_for_csv = df_text[["nonprofit_id", "topic_result", "description"]]
df_for_csv.to_csv(f"{datetime.now().strftime('%Y_%m_%d_%H_%M')}_unique_count_{len(df_text['nonprofit_id'].unique())}_nonprofit_text_output.csv", sep="|")

In [278]:
len(df_text["nonprofit_id"].unique())

4085

In [279]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.609321297084733

Coherence Score:  0.3735934994178092


In [280]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.284911  0.151437       1        1  32.264288
0      0.196143 -0.301248       2        1  25.925897
3      0.015449  0.227357       3        1  15.452972
4     -0.188699  0.014148       4        1   9.125875
5     -0.163788 -0.095767       5        1   8.643591
2     -0.144017  0.004074       6        1   8.587376, topic_info=          Term         Freq        Total Category  logprob  loglift
24     provide  2143.000000  2143.000000  Default  30.0000  30.0000
304     school   678.000000   678.000000  Default  29.0000  29.0000
60     service   969.000000   969.000000  Default  28.0000  28.0000
266     people   367.000000   367.000000  Default  27.0000  27.0000
41       child   689.000000   689.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
6          due    38.251056    39.014554   Topic6  -5.2308   2.4351
221      youth   186.276373   197.314676   Topic6  -3.6478   2.3973
691      world    99.029598   113.082440   Topic6  -4.2796   2.3222
392  establish    47.647458    54.769400   Topic6  -5.0112   2.3156
319  christian    82.349749   187.976541   Topic6  -4.4640   1.6295

[241 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
34        3  0.993630  academic
902       4  0.197405    access
902       5  0.789619    access
1         2  0.998533  activity
222       1  0.996963     adult
...     ...       ...       ...
32        1  0.131226      year
32        3  0.865270      year
535       1  0.989518     young
221       1  0.050680     youth
221       6  0.942657     youth

[237 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 5, 6, 3])